<h3>Importing Packages</h3>

In [1]:
import re
import bs4
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

<h3>Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe</h3>

In [10]:
toronto_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
bs = BeautifulSoup(toronto_data, 'lxml')

<h3>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</h3>

In [44]:
data = bs.find('table', class_="wikitable sortable")
df = pd.read_html(str(data))[0]
column_names = df.columns
list = df.drop([0])
list.columns = column_names
list.head()
new_list = list[-list.Borough.isin(["Not Assigned"])]
new_list.head()

,Postal Code,Borough,Neighbourhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"


<h3>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.</h3>

In [45]:
new_list['Neighbourhood'][new_list['Neighbourhood'] == "Not Assigned"] = new_list['Borough']
grouping = new_list.groupby(['Postal Code', 'Borough'], as_index = False).agg({'Neighbourhood': ['sum']})
grouping.head()

,Postal Code,Borough,Neighbourhood
,,,sum
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [47]:
print(grouping.columns.size)
type(grouping)

3


pandas.core.frame.DataFrame

<h3>Clean your Notebook</h3>

In [48]:
grouping.columns = column_names
grouping.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h3>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</h3>

In [49]:
for i in grouping['Neighbourhood']:
    ','.join(i)
print(grouping.head())
print(grouping.shape)

  Postal Code      Borough                           Neighbourhood
0         M1B  Scarborough                          Malvern, Rouge
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough       Guildwood, Morningside, West Hill
3         M1G  Scarborough                                  Woburn
4         M1H  Scarborough                               Cedarbrae
(179, 3)


<h1>Part 2</h1>

In [51]:
lat_long_coords = pd.read_csv("http://cocl.us/Geospatial_data", index_col = 0)

In [52]:
lat_long_coords.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [55]:
lat_long_coords.index

Index(['M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M', 'M1N',
       ...
       'M9A', 'M9B', 'M9C', 'M9L', 'M9M', 'M9N', 'M9P', 'M9R', 'M9V', 'M9W'],
      dtype='object', name='Postal Code', length=103)

In [58]:
group = lat_long_coords.sort_index(axis = 0, ascending = True)
group.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [61]:
group.shape

(103, 2)

In [62]:
group.tail()

,Latitude,Longitude
Postal Code,,
M9N,43.706876,-79.518188
M9P,43.696319,-79.532242
M9R,43.688905,-79.554724
M9V,43.739416,-79.588437
M9W,43.706748,-79.594054


<h1>Part 3</h1>

<h3>Grouping Neighbourhoods in Toronto</h3>

In [63]:
group_tor = grouping[grouping['Borough'].str.contains('Toronto')]
group_tor.head()

,Postal Code,Borough,Neighbourhood
62,M4E,East Toronto,The Beaches
66,M4K,East Toronto,"The Danforth West, Riverdale"
67,M4L,East Toronto,"India Bazaar, The Beaches West"
68,M4M,East Toronto,Studio District
69,M4N,Central Toronto,Lawrence Park
